In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

class SVM:
    def __init__(self, C=1.0, max_iter=100, kernel='linear'):
        self.C = C  # Regularization parameter
        self.max_iter = max_iter  # Maximum number of iterations
        self.kernel = kernel  # Kernel function ('linear' or 'rbf')
        self.kernel_func = self._linear_kernel if kernel == 'linear' else self._rbf_kernel
        
    def fit(self, X, y):
        self.X_train = X  # Store training data
        self.y_train = y  # Store training labels
        n_samples, n_features = X.shape
        
        # Initialize parameters
        self.alpha = np.zeros(n_samples)
        self.b = 0
        
        # Precompute kernel matrix
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            self.K[i] = self.kernel_func(X[i], X)
        
        # Training
        for _ in range(self.max_iter):
            for i in range(n_samples):
                # Pick random sample (i1) and compute prediction
                i1 = self._random_selection(i, n_samples)
                y_pred_i1 = self._predict(X[i1])
                
                # Compute error
                Ei1 = y_pred_i1 - y[i1]
                
                # Pick second sample (i2) using heuristics
                i2 = self._heuristic_selection(Ei1, i, n_samples)
                
                # Compute predictions
                y_pred_i2 = self._predict(X[i2])
                Ei2 = y_pred_i2 - y[i2]
                
                # Save old alphas
                alpha_i1_old = self.alpha[i1]
                alpha_i2_old = self.alpha[i2]
                
                # Compute L and H
                if y[i1] != y[i2]:
                    L = max(0, self.alpha[i2] - self.alpha[i1])
                    H = min(self.C, self.C + self.alpha[i2] - self.alpha[i1])
                else:
                    L = max(0, self.alpha[i1] + self.alpha[i2] - self.C)
                    H = min(self.C, self.alpha[i1] + self.alpha[i2])
                
                # Update alpha i2
                self.alpha[i2] = alpha_i2_old + y[i2] * (Ei1 - Ei2) / (self.K[i1, i1] + self.K[i2, i2] - 2 * self.K[i1, i2])
                
                # Clip alpha i2
                self.alpha[i2] = max(L, min(H, self.alpha[i2]))
                
                # Update alpha i1
                self.alpha[i1] = alpha_i1_old + y[i1] * y[i2] * (alpha_i2_old - self.alpha[i2])
                
                # Update threshold b
                b1 = self.b - Ei1 - y[i1] * (self.alpha[i1] - alpha_i1_old) * self.K[i1, i1] - y[i2] * (self.alpha[i2] - alpha_i2_old) * self.K[i1, i2]
                b2 = self.b - Ei2 - y[i1] * (self.alpha[i1] - alpha_i1_old) * self.K[i1, i2] - y[i2] * (self.alpha[i2] - alpha_i2_old) * self.K[i2, i2]
                
                if 0 < self.alpha[i1] < self.C:
                    self.b = b1
                elif 0 < self.alpha[i2] < self.C:
                    self.b = b2
                else:
                    self.b = (b1 + b2) / 2
    
    def _predict(self, X):
        return np.sign(np.sum(self.alpha * self.kernel_func(X, self.X_train)) + self.b)
    
    def predict(self, X):
        return np.array([self._predict(x) for x in X])
    
    def _linear_kernel(self, x1, x2):
        return np.dot(x1, x2.T)
    
    def _rbf_kernel(self, x1, x2, gamma=0.1):
        return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)
    
    def _random_selection(self, i, n_samples):
        i1 = i
        while i1 == i:
            i1 = np.random.randint(n_samples)
        return i1
    
    def _heuristic_selection(self, Ei1, i, n_samples):
        i2 = -1
        max_delta_error = 0
        for j in range(n_samples):
            if j != i:
                Ej = self._predict(self.X_train[j]) - self.y_train[j]
                delta_error = abs(Ei1 - Ej)
                if delta_error > max_delta_error:
                    i2 = j
                    max_delta_error = delta_error
        if i2 == -1:  # if i2 is not updated, choose randomly
            i2 = self._random_selection(i, n_samples)
        return i2

# Load the dataset
data = pd.read_csv("Social_Network_Ads.csv")
one = LabelEncoder()
data['Gender'] = one.fit_transform(data['Gender'])
# Separate features and target variable
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train SVM
svm = SVM(C=1.0, max_iter=100, kernel='rbf')
svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.35
Precision: 0.35
Recall: 1.0
F1 Score: 0.5185185185185185
ROC AUC Score: 0.5
Confusion Matrix:
 [[ 0 52]
 [ 0 28]]
